<a href="https://colab.research.google.com/github/tuitenrein/Google-Colab/blob/main/IEC_Task_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, classification_report
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import GridSearchCV 
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC



df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data_benign_sms_equal.csv')
label_encoder = LabelEncoder()
df['Label'] = label_encoder.fit_transform(df['Label'])
df

,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,3220,10.42.0.211-123.125.115.164-48687-443-6,10.42.0.211,48687.0,123.125.115.164,443.0,6.0,05/08/2017 04:58:37,4699333.0,3.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,1052120,10.42.0.42-58.63.236.212-53777-443-6,10.42.0.42,53777.0,58.63.236.212,443.0,6.0,30/06/2017 07:37:35,30602670.0,2.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,1299883,183.79.250.251-10.42.0.211-443-54369-6,10.42.0.211,54369.0,183.79.250.251,443.0,6.0,04/08/2017 05:54:13,11036953.0,11.0,...,32.0,897540.0,0.0,897540.0,897540.0,10139356.0,0.0,10139356.0,10139356.0,0
3,717454,10.42.0.151-104.244.43.227-53259-443-6,10.42.0.151,53259.0,104.244.43.227,443.0,6.0,05/07/2017 05:44:09,3815335.0,3.0,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,129803,10.42.0.211-10.42.0.1-61002-53-17,10.42.0.211,61002.0,10.42.0.1,53.0,17.0,05/08/2017 04:47:07,510550.0,1.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77904,800989,10.42.0.1-10.42.0.42-53-35386-17,10.42.0.42,35386.0,10.42.0.1,53.0,17.0,17/08/2017 11:48:01,49480.0,1.0,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12
77905,808707,10.42.0.42-64.71.142.95-45060-443-6,10.42.0.42,45060.0,64.71.142.95,443.0,6.0,17/08/2017 11:48:52,30150466.0,6.0,...,20.0,294812.0,0.0,294812.0,294812.0,29855632.0,0.0,29855632.0,29855632.0,12
77906,748781,10.42.0.42-104.112.242.202-33285-80-6,10.42.0.42,33285.0,104.112.242.202,80.0,6.0,17/08/2017 11:24:13,100144.0,5.0,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12
77907,1119914,172.217.10.138-10.42.0.42-443-58098-6,172.217.10.138,443.0,10.42.0.42,58098.0,6.0,16/08/2017 04:05:51,20.0,2.0,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12


In [ ]:
X,y = df.drop('Label', axis=1), df.Label
X = X.select_dtypes(include='number')
X

,Unnamed: 0,Source Port,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,3220,48687.0,443.0,6.0,4699333.0,3.0,0.0,0.0,0.0,0.0,...,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1052120,53777.0,443.0,6.0,30602670.0,2.0,2.0,0.0,0.0,0.0,...,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1299883,54369.0,443.0,6.0,11036953.0,11.0,14.0,817.0,9903.0,474.0,...,3.0,32.0,897540.0,0.0,897540.0,897540.0,10139356.0,0.0,10139356.0,10139356.0
3,717454,53259.0,443.0,6.0,3815335.0,3.0,0.0,0.0,0.0,0.0,...,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,129803,61002.0,53.0,17.0,510550.0,1.0,1.0,34.0,126.0,34.0,...,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77904,800989,35386.0,53.0,17.0,49480.0,1.0,1.0,37.0,182.0,37.0,...,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
77905,808707,45060.0,443.0,6.0,30150466.0,6.0,8.0,328.0,3332.0,202.0,...,2.0,20.0,294812.0,0.0,294812.0,294812.0,29855632.0,0.0,29855632.0,29855632.0
77906,748781,33285.0,80.0,6.0,100144.0,5.0,4.0,419.0,1673.0,419.0,...,1.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
77907,1119914,443.0,58098.0,6.0,20.0,2.0,0.0,55.0,0.0,55.0,...,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
normalized_df = X / X.mean()

vt = VarianceThreshold(threshold=0.003)
_ = vt.fit(normalized_df)

mask = vt.get_support()

X_reduced = X.loc[:, mask]
X_reduced.shape

/usr/local/lib/python3.8/dist-packages/sklearn/feature_selection/_variance_threshold.py:104: RuntimeWarning: Degrees of freedom <= 0 for slice.
  self.variances_ = np.nanvar(X, axis=0)


(77909, 69)

In [ ]:
def identify_correlated(df, threshold):
    """
    A function to identify highly correlated features.
    """
    # Compute correlation matrix with absolute values
    matrix = df.corr().abs()

    # Create a boolean mask
    mask = np.triu(np.ones_like(matrix, dtype=bool))

    # Subset the matrix
    reduced_matrix = matrix.mask(mask)

    # Find cols that meet the threshold
    to_drop = [c for c in reduced_matrix.columns if any(reduced_matrix[c] > threshold)]

    return to_drop

In [ ]:
to_drop = identify_correlated(X_reduced, threshold=0.9)
X_reduced.drop(to_drop, axis=1, inplace=True)

rfecv = RFECV(
    estimator=DecisionTreeClassifier(),
    scoring="r2",
    cv=3,
    n_jobs=-1,
    min_features_to_select=5,
    step=5,
)

print(X_reduced.shape)
_ = rfecv.fit(X_reduced, y)
mask = rfecv.support_

X_final = X_reduced.loc[:, mask].copy()

(77909, 40)


/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_final, y, test_size=0.3)

In [ ]:
# Decision Tree Classifier
modelTree = DecisionTreeClassifier()
modelTree.fit(X_train, y_train)

DecisionTreeClassifier()

In [ ]:
y_pred = modelTree.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[ 223,   87,  135,  208,  144,  136,  130,  104,  113,  154,  139,
         124,  116],
       [  76, 1123,   58,   76,  112,   81,   63,   48,   29,   67,   52,
          68,   27],
       [ 120,   57,  285,  112,   80,   78,  325,  308,   66,   70,  111,
         121,   48],
       [ 166,  106,  125,  300,  132,  132,  140,  143,   94,  138,  121,
         131,   76],
       [ 128,   91,   98,  130,  339,  164,  424,   87,   77,  112,   58,
          63,   53],
       [  98,   82,   86,  119,  143,  530,  109,  303,   41,  108,   69,
          60,   36],
       [ 106,   64,  355,   94,  388,   93,  212,   82,   62,  101,  100,
          83,   32],
       [ 104,   46,  318,   87,   79,  305,   91,  445,   43,   72,   85,
          83,   32],
       [ 129,   40,   52,   81,   60,   50,   62,   35,  717,  121,   54,
          53,  314],
       [ 157,   79,   88,  147,   94,   94,   96,   82,  126,  451,  139,
         130,  102],
       [ 144,   75,  142,  111,   88,   80,  102, 

In [ ]:
df_f1 = f1_score(y_test, y_pred, average='macro')
df_f1

0.26903087505695766

In [ ]:
dt_accuracy = accuracy_score(y_test, y_pred)
dt_accuracy

0.27061138920977196

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.13      0.12      0.13      1813
           1       0.58      0.60      0.59      1880
           2       0.15      0.16      0.15      1781
           3       0.18      0.17      0.17      1804
           4       0.19      0.19      0.19      1824
           5       0.29      0.30      0.29      1784
           6       0.11      0.12      0.12      1772
           7       0.24      0.25      0.24      1790
           8       0.40      0.41      0.40      1768
           9       0.26      0.25      0.26      1785
          10       0.24      0.22      0.23      1792
          11       0.26      0.26      0.26      1784
          12       0.47      0.47      0.47      1796

    accuracy                           0.27     23373
   macro avg       0.27      0.27      0.27     23373
weighted avg       0.27      0.27      0.27     23373



In [ ]:
# Random Forest Classifier
rf_model = RandomForestClassifier().fit(X_train, y_train)

In [ ]:
y_pred_rf = rf_model.predict(X_test)
confusion_matrix(y_test, y_pred_rf)

array([[ 258,   99,  117,  168,  136,  184,   96,  138,  105,  166,  140,
         134,   72],
       [  51, 1356,   44,   35,   75,   85,   41,   43,   21,   42,   26,
          31,   30],
       [  83,   60,  280,   95,   69,   80,  394,  370,   43,   52,   94,
         125,   36],
       [ 167,  116,  140,  368,  108,  135,   83,  143,   90,  157,  108,
         123,   66],
       [ 106,  101,   78,  103,  378,  195,  470,   90,   61,   95,   61,
          41,   45],
       [  76,  103,   59,   92,  131,  692,   61,  355,   34,   57,   47,
          45,   32],
       [  90,   82,  452,   78,  486,  102,  165,   64,   42,   66,   55,
          66,   24],
       [  72,   51,  350,   87,   77,  380,   47,  490,   26,   46,   67,
          80,   17],
       [  86,   23,   29,   55,   48,   63,   50,   34,  886,  117,   48,
          33,  296],
       [ 149,   93,   84,  140,  110,  141,   69,   68,  121,  511,  101,
         109,   89],
       [ 149,   68,  136,  141,   53,   65,   71, 

In [ ]:
rf_accuracy = accuracy_score(y_test, y_pred_rf)
rf_accuracy

0.30868951354126556

In [ ]:
rf_f1 = f1_score(y_test, y_pred_rf, average='macro')
rf_f1

0.3028742276070772

In [ ]:
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.17      0.14      0.16      1813
           1       0.60      0.72      0.66      1880
           2       0.14      0.16      0.15      1781
           3       0.24      0.20      0.22      1804
           4       0.22      0.21      0.21      1824
           5       0.31      0.39      0.35      1784
           6       0.10      0.09      0.10      1772
           7       0.24      0.27      0.26      1790
           8       0.46      0.50      0.48      1768
           9       0.32      0.29      0.30      1785
          10       0.28      0.25      0.26      1792
          11       0.31      0.29      0.30      1784
          12       0.52      0.48      0.50      1796

    accuracy                           0.31     23373
   macro avg       0.30      0.31      0.30     23373
weighted avg       0.30      0.31      0.30     23373



In [ ]:
# XGBoost Classifier
xgb_classifier = XGBClassifier()
xgb_classifier.fit(X_train, y_train)
y_pred = xgb_classifier.predict(X_test)

In [ ]:
confusion_matrix(y_test, y_pred)

array([[221,  88, 111,  91,  60, 411,  72, 225, 150,  81,  97, 166,  40],
       [116, 563,  68,  37,  64, 419,  71, 165, 103,  60,  53, 117,  44],
       [158,  55, 259,  54,  28, 316, 110, 345,  93,  58,  73, 203,  29],
       [113, 109, 168, 135,  63, 439,  38, 276,  99,  91,  70, 167,  36],
       [140, 124,  62,  46, 161, 582, 147, 243,  79,  56,  52, 101,  31],
       [ 30, 210,  90,  70,  77, 827,  18, 301,  27,  25,  36,  65,   8],
       [145,  75, 188,  43,  97, 417, 117, 225, 120,  74,  72, 158,  41],
       [ 48, 114, 177,  69,  48, 486,  26, 569,  54,  26,  48, 113,  12],
       [ 70,  24,  26,  50,  22, 255,  29, 113, 879,  96,  43,  68,  93],
       [157,  89,  84,  73,  52, 370,  38, 167, 162, 329, 108, 114,  42],
       [133,  68, 189,  59,  18, 266,  50, 225, 109,  99, 213, 333,  30],
       [111,  40, 184,  54,  16, 232,  57, 226, 103,  81, 195, 444,  41],
       [ 82,  40,  33,  45,  29, 218,  24, 119, 513,  88,  49,  54, 502]])

In [ ]:
xgb_accuracy = accuracy_score(y_test, y_pred)
xgb_accuracy

0.22329183245625295

In [ ]:
xgb_f1 = f1_score(y_test, y_pred, average='macro')
xgb_f1

0.21290406768276174

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.15      0.12      0.13      1813
           1       0.35      0.30      0.32      1880
           2       0.16      0.15      0.15      1781
           3       0.16      0.07      0.10      1804
           4       0.22      0.09      0.13      1824
           5       0.16      0.46      0.24      1784
           6       0.15      0.07      0.09      1772
           7       0.18      0.32      0.23      1790
           8       0.35      0.50      0.41      1768
           9       0.28      0.18      0.22      1785
          10       0.19      0.12      0.15      1792
          11       0.21      0.25      0.23      1784
          12       0.53      0.28      0.37      1796

    accuracy                           0.22     23373
   macro avg       0.24      0.22      0.21     23373
weighted avg       0.24      0.22      0.21     23373



In [ ]:
# Naïve Bayes Classifier
model = MultinomialNB()
model.fit(X_train.abs(), y_train)

MultinomialNB()

In [ ]:
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[  66,  202,    3,   40,    0,    2,    0,    0,   53,    4,   27,
         192, 1224],
       [  90,  293,    2,   34,    0,    3,    0,    2,   60,    3,   14,
         218, 1161],
       [  73,  161,    5,   31,    0,    1,    0,    4,   57,    3,   22,
         197, 1227],
       [  76,  244,    2,   37,    0,    2,    0,    6,   63,    6,   27,
         201, 1140],
       [  71,  216,    2,   46,    1,    4,    0,    5,   66,    2,   20,
         191, 1200],
       [ 107,  303,    1,   42,    0,    2,    0,    7,   25,    6,   27,
         280,  984],
       [  55,  187,    4,   26,    0,    2,    0,    4,   61,    3,   14,
         213, 1203],
       [ 114,  272,    4,   36,    0,    1,    0,    3,   25,    6,   24,
         253, 1052],
       [  62,  134,    3,   27,    0,    5,    0,    4,  134,    4,   19,
         136, 1240],
       [  63,  185,    2,   30,    0,    1,    0,    0,   83,   14,   17,
         193, 1197],
       [  49,  146,    3,   35,    0,    6,    0, 

In [ ]:
nb_f1 = f1_score(y_test, y_pred, average='weighted')
nb_f1

0.047531113649239204

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.07      0.04      0.05      1813
           1       0.11      0.16      0.13      1880
           2       0.15      0.00      0.01      1781
           3       0.08      0.02      0.03      1804
           4       1.00      0.00      0.00      1824
           5       0.03      0.00      0.00      1784
           6       0.00      0.00      0.00      1772
           7       0.07      0.00      0.00      1790
           8       0.16      0.08      0.10      1768
           9       0.26      0.01      0.02      1785
          10       0.08      0.01      0.02      1792
          11       0.08      0.11      0.09      1784
          12       0.09      0.79      0.16      1796

    accuracy                           0.09     23373
   macro avg       0.17      0.09      0.05     23373
weighted avg       0.17      0.09      0.05     23373



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Support Vector Machines (SVM)

svc_model = SVC()
svc_model.fit(X_train, y_train)

SVC()

In [ ]:
y_pred  = svc_model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[  12,   43,   23,    7,    4,  194,    2,   23,   19,    2,   41,
           8, 1435],
       [   8,  105,    6,    7,    1,  392,    0,   22,   32,    0,   38,
           0, 1269],
       [   3,   77,   18,    5,    6,  149,    0,   23,   16,    3,   40,
           4, 1437],
       [   5,   65,   19,    9,    2,  239,    0,   32,   33,    2,   35,
          12, 1351],
       [   5,  102,    7,    7,    1,  302,    0,   34,   18,    0,   35,
           6, 1307],
       [  15,   98,   13,   10,    5,  406,    0,   24,   10,    0,   20,
           5, 1178],
       [   6,   77,   16,   12,    6,  212,    0,   29,   19,    1,   41,
           6, 1347],
       [   9,   82,   36,    4,    2,  251,    0,   41,   19,    2,   51,
           4, 1289],
       [   4,   43,    6,    4,    2,  152,    0,   43,  112,    2,   26,
          10, 1364],
       [  10,   55,    6,    6,    1,  246,    0,   23,   46,    6,   44,
          10, 1332],
       [   8,   65,   26,    2,    1,  133,    0, 

In [ ]:
svm_f1 = f1_score(y_test, y_pred, average='weighted')
svm_f1

0.048925934685828204

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.12      0.01      0.01      1813
           1       0.12      0.06      0.08      1880
           2       0.09      0.01      0.02      1781
           3       0.11      0.00      0.01      1804
           4       0.03      0.00      0.00      1824
           5       0.14      0.23      0.17      1784
           6       0.00      0.00      0.00      1772
           7       0.11      0.02      0.04      1790
           8       0.26      0.06      0.10      1768
           9       0.33      0.00      0.01      1785
          10       0.06      0.02      0.02      1792
          11       0.12      0.01      0.01      1784
          12       0.09      0.87      0.16      1796

    accuracy                           0.10     23373
   macro avg       0.12      0.10      0.05     23373
weighted avg       0.12      0.10      0.05     23373



In [ ]:
# AdaBoost
model = AdaBoostClassifier()
model.fit(X_train, y_train)

AdaBoostClassifier()

In [ ]:
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[100,  41,  59,  88,  44, 368,  38, 426, 178,  43,  90, 260,  78],
       [ 69, 141,  40,  87,  66, 521,  20, 366, 156,  73,  66, 207,  68],
       [ 68,  50,  74,  49,  22, 238,  33, 601, 178,  39,  93, 272,  64],
       [ 73,  44,  48, 107,  60, 375,  27, 529, 146,  55,  67, 232,  41],
       [ 93,  82,  66,  99,  99, 431,  37, 423, 139,  57,  46, 186,  66],
       [ 45,  79,  42, 100,  82, 609,   4, 570,  64,  73,  23,  92,   1],
       [ 86,  55,  55,  74,  48, 328,  51, 435, 170,  45,  62, 263, 100],
       [ 96,  57,  46,  77,  64, 419,  11, 715, 104,  27,  50, 121,   3],
       [ 44,  14,  61,  61,  33, 186,  23, 242, 648,  40,  31, 201, 184],
       [ 85,  38,  44, 105,  64, 317,  32, 338, 160,  88, 157, 247, 110],
       [ 69,  43,  32, 111,  26, 285,  35, 459, 125,  58, 153, 344,  52],
       [ 54,  38,  48,  88,  19, 238,  29, 480, 137,  37, 161, 388,  67],
       [ 54,  29,  37,  84,  45, 139,  19, 184, 473,  38,  42, 183, 469]])

In [ ]:
ada_f1 = f1_score(y_test, y_pred, average='weighted')
ada_f1

0.13532382505720889

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.11      0.06      0.07      1813
           1       0.20      0.07      0.11      1880
           2       0.11      0.04      0.06      1781
           3       0.09      0.06      0.07      1804
           4       0.15      0.05      0.08      1824
           5       0.14      0.34      0.20      1784
           6       0.14      0.03      0.05      1772
           7       0.12      0.40      0.19      1790
           8       0.24      0.37      0.29      1768
           9       0.13      0.05      0.07      1785
          10       0.15      0.09      0.11      1792
          11       0.13      0.22      0.16      1784
          12       0.36      0.26      0.30      1796

    accuracy                           0.16     23373
   macro avg       0.16      0.16      0.14     23373
weighted avg       0.16      0.16      0.14     23373

